#Kmeans clustering with number of events in minute for specific session id (derived column)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
pd.set_option('display.max_rows', None)

In [9]:
pd.set_option('display.max_columns', 500)

In [10]:
df = pd.read_parquet("/content/drive/MyDrive/datathon - final/full_data.parquet")

In [11]:
df.head()

,event_id,session_id,transaction_date,event_type_id,product_id,title_product_id,country_code_id,ip_address,auth_session_id
0,0,3507032552,2020-08-20,4,63795519,63077424,162,238.108.111.146,2668673246
1,1,3506994387,2020-08-20,5,193402914,63110659,148,32.129.93.188,2668641289
2,2,3507038875,2020-08-20,5,202835637,63108769,517,222.60.132.210,2668676616
3,3,3507038876,2020-08-20,7,63109557,63109558,351,22.28.25.247,2668676617
4,4,3507036177,2020-08-20,4,69817018,63129191,162,138.169.123.254,2668677577


In [12]:
df.shape

(6044797, 9)

In [9]:
df.info(null_counts=True)

<ipython-input-9-acd3301a9435>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6044797 entries, 0 to 6044796
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   event_id          6044797 non-null  int64         
 1   session_id        6044797 non-null  object        
 2   transaction_date  6044797 non-null  datetime64[ns]
 3   event_type_id     6043230 non-null  object        
 4   product_id        6044797 non-null  int64         
 5   title_product_id  6044797 non-null  int64         
 6   country_code_id   6044462 non-null  object        
 7   ip_address        6044797 non-null  object        
 8   auth_session_id   5958585 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 415.1+ MB


In [5]:
df = pd.read_parquet('/content/drive/MyDrive/datathon - final/df_date.parquet')

In [6]:
df.head()

,event_id,session_id,transaction_date,event_type_id,product_id,title_product_id,country_code_id,ip_address,auth_session_id,hour,minute,second
0,0,3507032552,2020-08-20,4,63795519,63077424,162,238.108.111.146,2668673246,0,0,0
1,1,3506994387,2020-08-20,5,193402914,63110659,148,32.129.93.188,2668641289,0,0,0
2,2,3507038875,2020-08-20,5,202835637,63108769,517,222.60.132.210,2668676616,0,0,0
3,3,3507038876,2020-08-20,7,63109557,63109558,351,22.28.25.247,2668676617,0,0,0
4,4,3507036177,2020-08-20,4,69817018,63129191,162,138.169.123.254,2668677577,0,0,0


In [7]:
for col in df:
    print(col, df[col].nunique())

event_id 6044797
session_id 3586748
transaction_date 86400
event_type_id 26
product_id 2008439
title_product_id 21187
country_code_id 312
ip_address 1130066
auth_session_id 3213231
hour 24
minute 60
second 60


In [8]:
df_fe = df.copy()

In [9]:
df_fe = df_fe.drop(columns=['event_id', 'transaction_date'])

In [10]:
df_gpby = df.groupby(['session_id', 'hour', 'minute'])['minute'].count()

In [11]:
df_gpby.head()

session_id  hour  minute
3503162006  1     8         1
3505498857  0     1         1
                  6         1
                  12        1
                  17        1
Name: minute, dtype: int64

In [12]:
df_gpby = df_gpby.reset_index(name="count")
df_gpby


KeyboardInterrupt



In [17]:
df_gpby.head()

,session_id,hour,minute,count
0,3503162006,1,8,1
1,3505498857,0,1,1
2,3505498857,0,6,1
3,3505498857,0,12,1
4,3505498857,0,17,1


In [18]:
df_fe_mer = df_fe.merge(df_gpby, on=['session_id', 'hour', 'minute'],  how='left')

In [14]:
df_fe_mer.head()

,session_id,event_type_id,product_id,title_product_id,country_code_id,ip_address,auth_session_id,hour,minute,second,count
0,3507032552,4,63795519,63077424,162,238.108.111.146,2668673246,0,0,0,1
1,3506994387,5,193402914,63110659,148,32.129.93.188,2668641289,0,0,0,12
2,3507038875,5,202835637,63108769,517,222.60.132.210,2668676616,0,0,0,1
3,3507038876,7,63109557,63109558,351,22.28.25.247,2668676617,0,0,0,1
4,3507036177,4,69817018,63129191,162,138.169.123.254,2668677577,0,0,0,1


In [13]:
# df_fe_mer.to_parquet('/content/drive/MyDrive/datathon - final/df_fe_mer.parquet', index=False)
df_fe_mer = pd.read_parquet('df_fe_mer.parquet')

In [15]:
from sklearn.preprocessing import LabelEncoder
for col in ['session_id', 'event_type_id', 'product_id', 'title_product_id', 'country_code_id', 'ip_address', 'auth_session_id']:
  le = LabelEncoder()
  df_fe_mer[col] = le.fit_transform(df_fe_mer[col])

In [16]:
df_fe_mer.head()

,session_id,event_type_id,product_id,title_product_id,country_code_id,ip_address,auth_session_id,hour,minute,second,count
0,2375,10,138042,224,52,699319,676559,0,0,0,1
1,1263,12,1355664,8771,41,776508,675855,0,0,0,12
2,4053,12,1944828,8157,286,621133,677534,0,0,0,1
3,4054,14,17776,8416,182,596770,677535,0,0,0,1
4,3074,10,1060630,12777,52,221310,677946,0,0,0,1


In [23]:
# df_fe_mer.to_parquet('/content/drive/MyDrive/datathon - final/df_fe_mer_encoded.parquet', index=False)
# df_fe_mer.to_parquet('df_fe_mer.parquet', index=False)
df_fe_mer = pd.read_parquet('/content/drive/MyDrive/datathon - final/df_fe_mer_encoded.parquet')

In [17]:
cols = df_fe_mer.columns

In [18]:
from sklearn.preprocessing import MinMaxScaler

ms = MinMaxScaler()

X = ms.fit_transform(df_fe_mer)

X = pd.DataFrame(X, columns=[cols])

In [19]:
X.head()

,session_id,event_type_id,product_id,title_product_id,country_code_id,ip_address,auth_session_id,hour,minute,second,count
0,0.000662,0.384615,0.068731,0.010573,0.166667,0.618831,0.210554,0.0,0.0,0.0,0.000000
1,0.000352,0.461538,0.674984,0.414000,0.131410,0.687136,0.210335,0.0,0.0,0.0,0.032641
2,0.001130,0.461538,0.968329,0.385018,0.916667,0.549644,0.210858,0.0,0.0,0.0,0.000000
3,0.001130,0.538462,0.008851,0.397243,0.583333,0.528085,0.210858,0.0,0.0,0.0,0.000000
4,0.000857,0.384615,0.528087,0.603087,0.166667,0.195838,0.210986,0.0,0.0,0.0,0.000000


# Kmeans Clustering

In [20]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0) 

kmeans.fit(X)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


KMeans(n_clusters=2, random_state=0)

In [21]:
kmeans.cluster_centers_

array([[0.27787607, 0.29577105, 0.55738584, 0.3177363 , 0.41934966,
        0.47974702, 0.26102145, 0.28723233, 0.49989074, 0.50115399,
        0.0059067 ],
       [0.74638191, 0.29678874, 0.50160449, 0.32368246, 0.45717624,
        0.46900016, 0.64557653, 0.74499629, 0.50341944, 0.50093943,
        0.00378796]])

In [22]:
kmeans.inertia_

4028296.1357299415

In [23]:
labels = kmeans.labels_

In [24]:
np.size(labels)

6044797

In [25]:
unique, counts = np.unique(labels, return_counts=True)

dict(zip(unique, counts))

{0: 3349431, 1: 2695366}

In [26]:
labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

# check the results of kmeans with session id which has more ip addresses

In [27]:
df_result = pd.DataFrame()
df_result['event_id'] = df['event_id']
df_result['crawler_flag'] = labels

In [28]:
# df_result.to_parquet('/content/drive/MyDrive/datathon - final/df_result2.parquet', index=False)

In [14]:
sid_list = []
for sid in df['session_id'].unique():
  if df[df['session_id'] == sid]['ip_address'].nunique() > 1:
    sid_list.append(sid)
    print(sid)

3506402690
3506906504
3507029629
3507028359
3507038161
3507017441
3507030766
3506935799
3507038909
3506930790
3506996723
3507039821
3507037916
3507038957
3506985527
3507039717
3507039868
3507031940
3507010158
3506911327
3507039681
3507039973
3507036931
3506968018
3507029964
3507040838
3507040858
3507038447
3507030932
3507027873
3506961621
3507040092
3507034639
3507026463
3507040951
3507040146
3507043769
3506898321
3507037651
3507044792
3507035692
3507037655
3507042972
3507043887
3507043882
3507038638
3507042097


KeyboardInterrupt: ignored

In [16]:
result = pd.read_parquet('/content/drive/MyDrive/datathon - final/df_result2.parquet')
result.head()

,event_id,crawler_flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [22]:
result[result['event_id']==72930]

,event_id,crawler_flag
72930,72930,0


In [25]:
event_ids = df.loc[df['session_id'].isin(sid_list)]['event_id'].values

In [28]:
result.loc[result['event_id'].isin(event_ids)]['crawler_flag'].value_counts()

0    727
Name: crawler_flag, dtype: int64

In [19]:
df[df['session_id']=='3506996723']['event_id']

439        439
11597    11597
18583    18583
21803    21803
22341    22341
27585    27585
48462    48462
48646    48646
72132    72132
72930    72930
Name: event_id, dtype: int64

In [15]:
sid_list

['3506402690',
 '3506906504',
 '3507029629',
 '3507028359',
 '3507038161',
 '3507017441',
 '3507030766',
 '3506935799',
 '3507038909',
 '3506930790',
 '3506996723',
 '3507039821',
 '3507037916',
 '3507038957',
 '3506985527',
 '3507039717',
 '3507039868',
 '3507031940',
 '3507010158',
 '3506911327',
 '3507039681',
 '3507039973',
 '3507036931',
 '3506968018',
 '3507029964',
 '3507040838',
 '3507040858',
 '3507038447',
 '3507030932',
 '3507027873',
 '3506961621',
 '3507040092',
 '3507034639',
 '3507026463',
 '3507040951',
 '3507040146',
 '3507043769',
 '3506898321',
 '3507037651',
 '3507044792',
 '3507035692',
 '3507037655',
 '3507042972',
 '3507043887',
 '3507043882',
 '3507038638',
 '3507042097']

In [ ]:
df.loc[df['column_name'].isin(some_values)]